In [1]:
import requests 
import pandas as pd

import numpy as np 

## Extract

In [12]:
#url for api

url = 'https://randomuser.me/api/'

num_of_users = '?results=100'

url = url + num_of_users

print(url)

https://randomuser.me/api/?results=100


In [14]:
response = requests.get(url)

In [15]:
response

<Response [200]>

In [16]:
if response.status_code == 200:
    data = response.json() 

In [17]:
data

{'results': [{'gender': 'male',
   'name': {'title': 'Mr', 'first': 'Vernon', 'last': 'Gonzalez'},
   'location': {'street': {'number': 6529, 'name': 'Queen Street'},
    'city': 'Lichfield',
    'state': 'West Sussex',
    'country': 'United Kingdom',
    'postcode': 'N28 7GD',
    'coordinates': {'latitude': '-12.2093', 'longitude': '-170.2160'},
    'timezone': {'offset': '+3:30', 'description': 'Tehran'}},
   'email': 'vernon.gonzalez@example.com',
   'login': {'uuid': 'f59aa7f0-5f2e-4421-9599-9d78635e7737',
    'username': 'organicmeercat617',
    'password': 'emperor',
    'salt': 'u7j4qlfZ',
    'md5': '084081b5b638e0ed6c5f9f2a8a832104',
    'sha1': '927f9682007dfd6490ea64b6bfd55e7f1ed82377',
    'sha256': 'd4b3380e6b0a82967d38b145f10ba1a0747b6217ed45dd9aab61d6094e2520a2'},
   'dob': {'date': '1991-10-06T17:28:15.166Z', 'age': 33},
   'registered': {'date': '2012-04-14T13:30:38.955Z', 'age': 12},
   'phone': '016977 3196',
   'cell': '07574 437080',
   'id': {'name': 'NINO', 'va

In [18]:
df = pd.json_normalize(data)

In [19]:
df

,results,info.seed,info.results,info.page,info.version
0,"[{'gender': 'male', 'name': {'title': 'Mr', 'f...",7ef412114231b105,100,1,1.4


In [20]:
for i in df['results']:
    print(i)

[{'gender': 'male', 'name': {'title': 'Mr', 'first': 'Vernon', 'last': 'Gonzalez'}, 'location': {'street': {'number': 6529, 'name': 'Queen Street'}, 'city': 'Lichfield', 'state': 'West Sussex', 'country': 'United Kingdom', 'postcode': 'N28 7GD', 'coordinates': {'latitude': '-12.2093', 'longitude': '-170.2160'}, 'timezone': {'offset': '+3:30', 'description': 'Tehran'}}, 'email': 'vernon.gonzalez@example.com', 'login': {'uuid': 'f59aa7f0-5f2e-4421-9599-9d78635e7737', 'username': 'organicmeercat617', 'password': 'emperor', 'salt': 'u7j4qlfZ', 'md5': '084081b5b638e0ed6c5f9f2a8a832104', 'sha1': '927f9682007dfd6490ea64b6bfd55e7f1ed82377', 'sha256': 'd4b3380e6b0a82967d38b145f10ba1a0747b6217ed45dd9aab61d6094e2520a2'}, 'dob': {'date': '1991-10-06T17:28:15.166Z', 'age': 33}, 'registered': {'date': '2012-04-14T13:30:38.955Z', 'age': 12}, 'phone': '016977 3196', 'cell': '07574 437080', 'id': {'name': 'NINO', 'value': 'GY 76 16 47 U'}, 'picture': {'large': 'https://randomuser.me/api/portraits/men/5

In [21]:
# Create lists to store extracted user data
first_name_ls = []
last_name_ls = []
emails_ls = []
genders_ls = []
countries_ls = []
age_ls = []

In [22]:
for user in data['results']:

    email = user.get('email', np.nan)
    emails_ls.append(email)

    first_name = user['name'].get('first', np.nan)
    first_name_ls.append(first_name)

    last_name = user['name'].get('last', np.nan)
    last_name_ls.append(last_name)

    gender = user.get('gender', np.nan)
    genders_ls.append(gender)

    country = user['location'].get('country', np.nan)
    countries_ls.append(country)

    age = user['dob'].get('age', np.nan)
    age_ls.append(age)

In [23]:
df = pd.DataFrame({
    'first_name': first_name_ls,
    'last_name': last_name_ls,
    'Email': emails_ls,
    'Gender':genders_ls,
    'Country':countries_ls,
    'age':age_ls
})

In [24]:
df.head()

,first_name,last_name,Email,Gender,Country,age
0,Vernon,Gonzalez,vernon.gonzalez@example.com,male,United Kingdom,33
1,Aleksa,Mladenović,aleksa.mladenovic@example.com,male,Serbia,43
2,Kristiane,Gentner,kristiane.gentner@example.com,female,Germany,39
3,Erin,Green,erin.green@example.com,female,Ireland,53
4,Arthur,Montgomery,arthur.montgomery@example.com,male,Ireland,70


# Load

In [25]:
import mysql.connector
mydb = mysql.connector.connect(host='localhost',user='root',passwd='nadeem123',database='users')

In [26]:
cursordb = mydb.cursor()

In [27]:
for index, row in df.iterrows():

    sql_query = 'insert into user_details (first_name,last_name,Email,Gender,Country,age) values (%s,%s,%s,%s,%s,%s)'
    cursordb.execute(sql_query, (row['first_name'],row['last_name'],row['Email'],row['Gender'],row['Country'],row['age']))
print('data loaded successfully')

mydb.commit()

data loaded successfully


### Create Stored Procedure in Sql to clean the data

In [28]:
# Calling Stored Procedure
cursordb.callproc('user_data_transformation')

()

In [29]:
mydb.commit() # your changes will be reflected after commit
mydb.close()